In [1]:
import tensorflow
from tensorflow import keras

2023-09-06 11:33:14.329804: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-06 11:33:14.401674: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
import os
tensorflow.config.list_physical_devices()
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [3]:
# from tensorflow.compat.v1 import ConfigProto
# from tensorflow.compat.v1 import InteractiveSession

# config=ConfigProto()
# config.gpu_options.allow_growth = True
# session = InteractiveSession(config=config)

import tensorflow as tf
gpu = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpu[0], True)

In [4]:
from Network import Generator, Discriminator
import matplotlib.pyplot as plt
plt.switch_backend('agg')
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.layers import UpSampling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
# import keras
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Lambda, Input

import skimage.transform
from skimage import data, io, filters
import numpy as np
from numpy import array
from skimage.transform import rescale, resize
# from scipy.misc import imresize
import os





In [5]:
np.random.seed(10)
shape_hr = (1200,9369,1)

def vgg_loss(y_true, y_pred):
    
    vgg19 = VGG19(include_top=False, weights='imagenet', input_shape=shape_hr)
    vgg19.trainable = False
    for l in vgg19.layers:
        l.trainable = False
    loss_model = Model(inputs=vgg19.input, outputs=vgg19.get_layer('block5_conv4').output)
    loss_model.trainable = False
    return K.mean(K.square(loss_model(y_true) - loss_model(y_pred)))

def get_gan_network(discriminator, shape, generator, optimizer):
    discriminator.trainable = False
    gan_input = Input(shape=shape)
    x = generator(gan_input)
    print(x.shape)
    gan_output = discriminator(x)
    gan = Model(inputs=gan_input, outputs=[x,gan_output])
    gan.compile(loss=[vgg_loss, "binary_crossentropy"],
                loss_weights=[1., 1e-3],
                optimizer=optimizer)

    return gan


def load_path(path):
    directories = []
    if os.path.isdir(path):
        directories.append(path)
    for elem in os.listdir(path):
        if os.path.isdir(os.path.join(path,elem)):
            directories = directories + load_path(os.path.join(path,elem))
            directories.append(os.path.join(path,elem))
    return directories

def normalize(input_data):

    return ((input_data.astype(np.float32) - 127.5)/127.5)
    
def load_data_from_dirs(dirs, ext):
    files = []
    file_names = []
    count = 0
    for d in dirs:
        for f in os.listdir(d): 
            if f.endswith(ext):
                image = io.imread(os.path.join(d,f))
#                 print("noice",image[:,:,0])
                if len(image.shape) > 2:
#                     print("lol")
                    files.append(image[:,:,0])
                    file_names.append(os.path.join(d,f))
                else:
                    files.append(image)
                    file_names.append(os.path.join(d,f))
                count = count + 1
    return files     
            
# def load_data_from_dirs_resize(dirs, ext, size):
#     files = []
#     file_names = []
#     count = 0
#     for d in dirs:
#         for f in os.listdir(d): 
#             if f.endswith(ext):
#                 files.append(resize(io.imread(os.path.join(d,f)), size))
#                 file_names.append(os.path.join(d,f))
#                 count = count + 1
#     return files     
                        
          
def load_data(directory, ext):

    files = load_data_from_dirs(load_path(directory), ext)
    return files


files = load_data("./resized_cropped_tmc/X_train", ".png")
x_train_lr = files
files = load_data("./resized_cropped_tmc/X_test", ".png")
x_test_lr = files
print("lol")
files = load_data("./ohrc_final_final/y_train", ".png")
x_train_hr = files
files = load_data("./ohrc_final_final/y_test", ".png")
x_test_hr = files

print("data loaded")


def hr_images(images):
    images_hr = array(images)
    return images_hr

# def lr_images(images_real , downscale):
    
#     images = []
#     for img in  range(len(images_real)):
#         images.append(imresize(images_real[img], [images_real[img].shape[0]//downscale,images_real[img].shape[1]//downscale], interp='bicubic', mode=None))
#     images_lr = array(images)
#     return images_lr

# def preprocess_HR(x):
#     return np.divide(x.astype(np.float32), 127.5) - np.ones_like(x,dtype=np.float32)


# def deprocess_HR(x):
#     input_data = (input_data + 1) * 127.5
#     return input_data.astype(np.uint8) 


# def preprocess_LR(x):
#     return np.divide(x.astype(np.float32), 255.)


# def deprocess_LR(x):
#     x = np.clip(x*255, 0, 255)
#     return x


    
def denormalize(input_data):
    input_data = (input_data + 1) * 127.5
    return input_data.astype(np.uint8) 

# def deprocess_LRS(x):
#     x = np.clip(x*255, 0, 255)
#     return x.astype(np.uint8)


x_train_hr = hr_images(x_train_hr)

x_train_lr = hr_images(x_train_lr)
# x_train_lr = normalize(x_train_lr)


x_test_hr = hr_images(x_test_hr)
# x_test_hr = normalize(x_test_hr)

x_test_lr = hr_images(x_test_lr)
# x_test_lr = normalize(x_test_lr)

print("data processed")

lol
data loaded
data processed


In [7]:
x_train_hr.shape

(524, 9369, 1200)

In [6]:
def plot_generated_images(epoch,generator, examples=3 , dim=(1, 3), figsize=(15, 5)):
    
    rand_nums = np.random.randint(0, x_test_hr.shape[0], size=examples)
    image_batch_hr = x_test_hr[rand_nums]
    image_batch_lr = x_test_lr[rand_nums]
    gen_img = generator.predict(image_batch_lr)
    generated_image = gen_img
#     image_batch_lr = image_batch_lr
    
    #generated_image = deprocess_HR(generator.predict(image_batch_lr))
    
    plt.figure(figsize=figsize)
    
    plt.subplot(dim[0], dim[1], 1)
    plt.imshow(image_batch_lr[1], interpolation='nearest')
    plt.axis('off')
        
    plt.subplot(dim[0], dim[1], 2)
    plt.imshow(generated_image[1], interpolation='nearest')
    plt.axis('off')
    
    plt.subplot(dim[0], dim[1], 3)
    plt.imshow(image_batch_hr[1], interpolation='nearest')
    plt.axis('off')
    
    plt.tight_layout()
    plt.savefig('output/gan_generated_image_epoch_%d.png' % epoch)
    

def train(epochs=1, batch_size=1):

    downscale_factor = 4
    
    batch_count = int(x_train_hr.shape[0] / batch_size)
    shape_lr = (65,369,1)
    
    generator = Generator(shape_lr).generator()
    discriminator = Discriminator(shape_hr).discriminator()

    adam = Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    generator.compile(loss=vgg_loss, optimizer=adam)
    discriminator.compile(loss="binary_crossentropy", optimizer=adam)
    
#     shape = (image_shape[0]//downscale_factor, image_shape[1]//downscale_factor, 3)
    shape_lr
    gan = get_gan_network(discriminator, shape_lr, generator, adam)

    for e in range(1, epochs+1):
        print ('-'*15, 'Epoch %d' % e, '-'*15)
        for _ in range(batch_count):
            
            rand_nums = np.random.randint(0, x_train_hr.shape[0], size=batch_size)
            
            image_batch_hr = x_train_hr[rand_nums]
            image_batch_lr = x_train_lr[rand_nums]
            generated_images_sr = generator.predict(image_batch_lr)

            real_data_Y = np.ones(batch_size) - np.random.random_sample(batch_size)*0.2
            fake_data_Y = np.random.random_sample(batch_size)*0.2
            
            discriminator.trainable = True
            
            d_loss_real = discriminator.train_on_batch(image_batch_hr, real_data_Y)
            d_loss_fake = discriminator.train_on_batch(generated_images_sr, fake_data_Y)
            #d_loss = 0.5 * np.add(d_loss_fake, d_loss_real)
            
            rand_nums = np.random.randint(0, x_train_hr.shape[0], size=batch_size)
            image_batch_hr = x_train_hr[rand_nums]
            image_batch_lr = x_train_lr[rand_nums]

            gan_Y = np.ones(batch_size) - np.random.random_sample(batch_size)*0.2
            discriminator.trainable = False
            loss_gan = gan.train_on_batch(image_batch_lr, [image_batch_hr,gan_Y])
            
        print("Loss HR , Loss LR, Loss GAN")
        print(d_loss_real, d_loss_fake, loss_gan)

        if e == 1 or e % 5 == 0:
            plot_generated_images(e, generator)
        if e % 300 == 0:
            generator.save('./output/gen_model%d.h5' % e)
            discriminator.save('./output/dis_model%d.h5' % e)
            gan.save('./output/gan_model%d.h5' % e)

train()

2023-09-06 11:34:37.250514: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-06 11:34:37.742246: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46889 MB memory:  -> device: 0, name: Quadro RTX 8000, pci bus id: 0000:9e:00.0, compute capability: 7.5


(None, 260, 1476, 3)


ValueError: Exception encountered when calling layer "model_1" (type Functional).

Input 0 of layer "conv2d_37" is incompatible with the layer: expected axis -1 of input shape to have value 1, but received input with shape (None, 260, 1476, 3)

Call arguments received by layer "model_1" (type Functional):
  • inputs=tf.Tensor(shape=(None, 260, 1476, 3), dtype=float32)
  • training=None
  • mask=None

In [ ]:
print("Num GPUs Available: ", len(tensorflow.config.list_physical_devices('GPU')))
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())